# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [62]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm, trange
tqdm.monitor_interval = 0
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [ ]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True) #第一次需要下载，之后将直接本地导入
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [60]:
# TODO:define model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet,self).__init__() # call the init function of father class
        self.conv1 = nn.Conv2d(1,64,3,1,1) # use 64 3*3 conv kernel, and the output is  64*28*28
        self.relu1 = nn.ReLU() # the first relu
        self.pool1 = nn.MaxPool2d(2,2) # the first pooling layer, the output is 64*14*14
        self.conv2 = nn.Conv2d(64,128,3,1,1) # the second conv layer, the output is 128*14*14
        self.relu2 = nn.ReLU() # the second relu
        self.pool2 = nn.MaxPool2d(2,2) # the second pooling layer, the output is 128*7*7

        self.dense1 = nn.Linear(7*7*128,1024) # the first fc layer, the output is 1*1024
        self.dropout = nn.Dropout(p=0.5) # avoid over-fitting
        self.dense2 = nn.Linear(1024,10) # the second fc layer, the output is 1*10
    
    # forward process
    def forward(self, input):
        input = self.conv1(input)
        input = self.relu1(input)
        input = self.pool1(input)
        input = self.conv2(input)
        input = self.relu2(input)
        input = self.pool2(input)
        input = input.view(input.size(0), -1) 
        input = self.dense1(input)
        input = self.dropout(input)
        final_output = self.dense2(input)

        return final_output

model = SimpleNet()


# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [61]:
# train and evaluate in each epoch
for epoch in  range(NUM_EPOCHS):
    # train
    # TODO:forward + backward + optimize
    for images, labels in tqdm(train_loader):
        preds = model(images)
        loss = criterion(preds, labels)
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    train_acc = 0
    test_acc = 0
    for images, labels in tqdm(train_loader):
        preds = model(images)
        rlts = torch.argmax(preds,1)
        train_acc += torch.sum(rlts == labels)
    for images, labels in tqdm(test_loader):
        preds = model(images)
        rlts = torch.argmax(preds,1)
        test_acc += torch.sum(rlts == labels)
    print("Epoch %d : train accuracy: %.2f%%"%(epoch+1, 100*train_acc/60000))
    print("Epoch %d : test accuracy: %.2f%%"%(epoch+1, 100*test_acc/10000))

100%|██████████| 78/78 [00:07<00:00,  9.95it/s]
Epoch 1 : train accuracy: 98.10%
Epoch 1 : test accuracy: 98.02%
100%|██████████| 78/78 [00:07<00:00,  9.93it/s]
Epoch 2 : train accuracy: 98.77%
Epoch 2 : test accuracy: 98.45%
100%|██████████| 78/78 [00:07<00:00, 10.07it/s]
Epoch 3 : train accuracy: 98.92%
Epoch 3 : test accuracy: 98.43%
100%|██████████| 78/78 [00:07<00:00, 10.10it/s]
Epoch 4 : train accuracy: 99.08%
Epoch 4 : test accuracy: 98.59%
100%|██████████| 78/78 [00:07<00:00, 10.06it/s]
Epoch 5 : train accuracy: 99.11%
Epoch 5 : test accuracy: 98.74%
100%|██████████| 78/78 [00:07<00:00,  9.96it/s]
Epoch 6 : train accuracy: 99.12%
Epoch 6 : test accuracy: 98.65%
100%|██████████| 78/78 [00:07<00:00, 10.12it/s]
Epoch 7 : train accuracy: 99.06%
Epoch 7 : test accuracy: 98.58%
100%|██████████| 78/78 [00:07<00:00, 10.06it/s]
Epoch 8 : train accuracy: 99.36%
Epoch 8 : test accuracy: 98.82%
100%|██████████| 78/78 [00:08<00:00,  9.41it/s]
Epoch 9 : train accuracy: 99.24%
Epoch 9 : test 

#### Q5:
Please print the training and testing accuracy.

In [66]:
# final evaluate
train_acc = 0
test_acc = 0
for images, labels in tqdm(train_loader):
    preds = model(images)
    rlts = torch.argmax(preds,1)
    train_acc += torch.sum(rlts == labels)
for images, labels in tqdm(test_loader):
    preds = model(images)
    rlts = torch.argmax(preds,1)
    test_acc += torch.sum(rlts == labels)
print("Final result : train accuracy: %.2f%%"%(100*train_acc/60000))
print("Final result : test accuracy: %.2f%%"%(100*test_acc/10000))

100%|██████████| 78/78 [00:09<00:00,  8.62it/s]
Final result : train accuracy: 99.38%
Final result : test accuracy: 98.66%
